In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/content/tmdb_5000_movies.csv')

In [3]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
df = df['original_title']

In [5]:
df.shape

(4803,)

In [6]:
movie_list = df.to_list()

In [7]:
movie_list[0:5]

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter']

Tokenization

In [8]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [9]:
tokenizer.fit_on_texts(movie_list)

In [10]:
seq = tokenizer.texts_to_sequences(movie_list)

In [11]:
seq[:10]

[[1564],
 [210, 2, 1, 431, 47, 432, 72],
 [1565],
 [1, 52, 211, 1566],
 [212, 601],
 [213, 8, 21],
 [1567],
 [902, 146, 2, 1568],
 [110, 214, 4, 1, 433, 53, 147],
 [173, 340, 261, 85, 2, 903]]

In [12]:
len(seq)

4803

In [13]:
tokenizer.word_index['the']

1

Creating the Training Dataset

In [14]:
x = []
y = []

drop_words = 0

for s in seq:
  if len(s) > 1:
    for idx in range(1,len(s)):
      temp_x = s[:idx]
      temp_y = s[idx]
      x.append(temp_x)
      y.append(temp_y)
  else:
    drop_words += 1

print("Total words dropped = ",drop_words)

Total words dropped =  1003


In [15]:
x[:15]

[[210],
 [210, 2],
 [210, 2, 1],
 [210, 2, 1, 431],
 [210, 2, 1, 431, 47],
 [210, 2, 1, 431, 47, 432],
 [1],
 [1, 52],
 [1, 52, 211],
 [212],
 [213],
 [213, 8],
 [902],
 [902, 146],
 [902, 146, 2]]

In [16]:
y[:15]

[2, 1, 431, 47, 432, 72, 52, 211, 1566, 601, 8, 21, 146, 2, 1568]

 Pad all the Sequences

In [17]:
x = tf.keras.preprocessing.sequence.pad_sequences(x)

In [18]:
x

array([[  0,   0,   0, ...,   0,   0, 210],
       [  0,   0,   0, ...,   0, 210,   2],
       [  0,   0,   0, ..., 210,   2,   1],
       ...,
       [  0,   0,   0, ...,   0,   0,  14],
       [  0,   0,   0, ...,   0,  14, 300],
       [  0,   0,   0, ...,  14, 300,  11]], dtype=int32)

In [19]:
x.shape

(8483, 14)

In [20]:
y = tf.keras.utils.to_categorical(y)

In [21]:
y.shape

(8483, 5045)

In [22]:
vocab_size = len(tokenizer.word_index)+1
vocab_size

5045

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,14),
    tf.keras.layers.LSTM(100,return_sequences=True),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(vocab_size,activation='softmax')
])

In [27]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 14)          70630     
                                                                 
 lstm_6 (LSTM)               (None, None, 100)         46000     
                                                                 
 lstm_7 (LSTM)               (None, 100)               80400     
                                                                 
 dense_6 (Dense)             (None, 100)               10100     
                                                                 
 dense_7 (Dense)             (None, 5045)              509545    
                                                                 
Total params: 716,675
Trainable params: 716,675
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

In [32]:
model.fit(x,y,epochs=1000,verbose=0)

In [33]:
model.save('nwp.h5')

In [34]:
vocab_array = np.array(list(tokenizer.word_index.keys()))

In [35]:
vocab_array

array(['the', 'of', 'a', ..., 'signed', 'sealed', 'delivered'],
      dtype='<U14')

In [56]:
len(vocab_array)

5044

In [73]:
def predict(text,n):
  for i in range(n):
    text_token = tokenizer.texts_to_sequences([text])
    text_padded = tf.keras.utils.pad_sequences(text_token,maxlen=14)
    temp = np.squeeze(model.predict(text_padded,verbose=0))
    AM= np.argmax(temp,axis=0)
    pred = AM
    pred = str(vocab_array[pred - 1])
    text += " " + pred
    
  return text

Making Predictions 

In [76]:
a = "Pirates"
predict(a,3)

'Pirates of the caribbean'

In [85]:
b = "The Amazing"
predict(b,2)

'The Amazing spider man'

In [97]:
c = "Harry Potter and the Prisoner"
predict(c,3)

'Harry Potter and the Prisoner of the titans'

In [100]:
d = "Mission"
predict(d,3)

'Mission impossible iii with'

In [101]:
e = "Clash of"
predict(e,2)

'Clash of the dead'

In [106]:
f = "Another"
predict(f,1)

'Another year'